In [1]:
import pandas as pd
import re
import numpy as np
from scipy import stats

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from dateutil import parser
from datetime import datetime

# Purpose

This notebook is to look for evidence for when leadership positions became vacant.

# Import Data

In [17]:
turnover = pd.read_csv("Turnover/Turnover_Physician_For_Merging.csv")
leadership = pd.read_csv("Leadership/Leadership_cleaned.csv")

### Filter down 

In [21]:
CoS_turnover= turnover[turnover["Specialty"] == "51 CHIEF OF STAFF"]
CoS_turnover.head()

,Unnamed: 0,Organization,Specialty,FY11,FY12,FY13,FY14,FY15,ID,Station,for_dups
43,217,(V01) (518) MC BEDFORD MA,51 CHIEF OF STAFF,NaN,1.0,NaN,NaN,NaN,NaN,518,518
92,266,(V01) (608) MC MANCHESTER NH,51 CHIEF OF STAFF,NaN,NaN,NaN,NaN,1.0,NaN,608,608
187,403,(V02) (528) HCS BUFFALO NY,51 CHIEF OF STAFF,NaN,NaN,NaN,0.24,NaN,528we,528,528we
302,562,(V03) (632) MC NORTHPORT L I NY,51 CHIEF OF STAFF,NaN,NaN,NaN,NaN,1.0,NaN,632,632
330,642,(V04) (529) MC BUTLER PA,51 CHIEF OF STAFF,1.0,NaN,NaN,NaN,NaN,NaN,529,529


In [22]:
when = []
for index, row in CoS_turnover.iterrows():
    if row["FY15"] == 1:
        when.append("FY15")
    elif row["FY14"]== 1:
        when.append("FY14")
    elif row["FY13"]== 1:
        when.append("FY13")
    elif row["FY12"]== 1:
        when.append("FY12")
    elif row["FY11"]== 1:
        when.append("FY11")
    else:
        when.append(None)
CoS_turnover["when"] = when
CoS_turnover.head()

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Unnamed: 0,Organization,Specialty,FY11,FY12,FY13,FY14,FY15,ID,Station,for_dups,when
43,217,(V01) (518) MC BEDFORD MA,51 CHIEF OF STAFF,NaN,1.0,NaN,NaN,NaN,NaN,518,518,FY12
92,266,(V01) (608) MC MANCHESTER NH,51 CHIEF OF STAFF,NaN,NaN,NaN,NaN,1.0,NaN,608,608,FY15
187,403,(V02) (528) HCS BUFFALO NY,51 CHIEF OF STAFF,NaN,NaN,NaN,0.24,NaN,528we,528,528we,None
302,562,(V03) (632) MC NORTHPORT L I NY,51 CHIEF OF STAFF,NaN,NaN,NaN,NaN,1.0,NaN,632,632,FY15
330,642,(V04) (529) MC BUTLER PA,51 CHIEF OF STAFF,1.0,NaN,NaN,NaN,NaN,NaN,529,529,FY11


In [23]:
CoS_turnover["when"].value_counts()

FY13    6
FY12    6
FY15    6
FY14    6
FY11    5
Name: when, dtype: int64

In [24]:
leadership.head()

,VISN,Station,Facility,Position,Acting/Detailed,StartDate,ID
0,1,402,"HCS Maine - Togus (Augusta, ME)",Assoc Med Ctr Director,VACANT,NaN,NaN
1,19,436,HCS Montana (Ft Harrison),Assoc Med Ctr Director,"HAMILTON, LISA",2/16/16,NaN
2,19,436,HCS Montana (Ft Harrison),Chief of Staff,"ATWATER, BENJAMIN",8/23/15,NaN
3,19,442,"VAMC Cheyenne, WY",Med Ctr Director,"ROBERTS, PAUL",10/1/15,NaN
4,21,459,"HCS Pacific Islands (Honolulu, HI)",Med Ctr Director,"BAGBY, TONIA",3/14/16,NaN


In [28]:
CoS_leadership = leadership[leadership["Position"]=="Chief of Staff"]
CoS_leadership.head()

,VISN,Station,Facility,Position,Acting/Detailed,StartDate,ID
2,19,436,HCS Montana (Ft Harrison),Chief of Staff,"ATWATER, BENJAMIN",8/23/15,NaN
8,18,501,HCS New Mexico (Albuquerque),Chief of Staff,"EMELITY, DAVID",4/1/16,NaN
14,11,506,"VAMC Ann Arbor, MI",Chief of Staff,"LYPSON, MONICA",3/6/16,NaN
16,8,516,"VAMC Bay Pines, FL",Chief of Staff,"PARK, HENRY",2/21/16,NaN
18,1,518,"VAMC Bedford, MA",Chief of Staff,VACANT,NaN,NaN


In [30]:
CoS_turnover

,Unnamed: 0,Organization,Specialty,FY11,FY12,FY13,FY14,FY15,ID,Station,for_dups,when
43,217,(V01) (518) MC BEDFORD MA,51 CHIEF OF STAFF,NaN,1.0000,NaN,NaN,NaN,NaN,518,518,FY12
92,266,(V01) (608) MC MANCHESTER NH,51 CHIEF OF STAFF,NaN,NaN,NaN,NaN,1.0,NaN,608,608,FY15
187,403,(V02) (528) HCS BUFFALO NY,51 CHIEF OF STAFF,NaN,NaN,NaN,0.2400,NaN,528we,528,528we,None
302,562,(V03) (632) MC NORTHPORT L I NY,51 CHIEF OF STAFF,NaN,NaN,NaN,NaN,1.0,NaN,632,632,FY15
330,642,(V04) (529) MC BUTLER PA,51 CHIEF OF STAFF,1.0000,NaN,NaN,NaN,NaN,NaN,529,529,FY11
348,660,(V04) (542) MC COATESVILLE PA,51 CHIEF OF STAFF,NaN,NaN,1.0,NaN,NaN,NaN,542,542,FY13
355,667,(V04) (562) MC ERIE PA,51 CHIEF OF STAFF,NaN,NaN,NaN,NaN,1.0,NaN,562,562,FY15
496,850,(V05) (512) HCS BALTIMORE MD,51 CHIEF OF STAFF,NaN,NaN,1.0,NaN,NaN,NaN,512,512,FY13
586,994,(V06) (558) MC DURHAM NC,51 CHIEF OF STAFF,NaN,NaN,NaN,NaN,0.5,NaN,558,558,None
884,1342,(V07) (544) MC COLUMBIA SC,51 CHIEF OF STAFF,NaN,NaN,1.0,NaN,NaN,NaN,544,544,FY13


In [33]:
def get_station(new_string):
    try:
        query = r"^\((V\d\d)\) \((.*)\)"
        m = re.search(query, new_string)
        return m.group(2).upper()
    except:
        None
CoS_turnover["Station"]=CoS_turnover["Organization"].apply(get_station)
CoS_turnover.head()

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Unnamed: 0,Organization,Specialty,FY11,FY12,FY13,FY14,FY15,ID,Station,for_dups,when
43,217,(V01) (518) MC BEDFORD MA,51 CHIEF OF STAFF,NaN,1.0,NaN,NaN,NaN,NaN,518,518,FY12
92,266,(V01) (608) MC MANCHESTER NH,51 CHIEF OF STAFF,NaN,NaN,NaN,NaN,1.0,NaN,608,608,FY15
187,403,(V02) (528) HCS BUFFALO NY,51 CHIEF OF STAFF,NaN,NaN,NaN,0.24,NaN,528we,528,528we,None
302,562,(V03) (632) MC NORTHPORT L I NY,51 CHIEF OF STAFF,NaN,NaN,NaN,NaN,1.0,NaN,632,632,FY15
330,642,(V04) (529) MC BUTLER PA,51 CHIEF OF STAFF,1.0,NaN,NaN,NaN,NaN,NaN,529,529,FY11


In [35]:
CoS_leadership.merge(CoS_turnover[["Station","when"]],on="Station")

,VISN,Station,Facility,Position,Acting/Detailed,StartDate,ID,when
0,1,518,"VAMC Bedford, MA",Chief of Staff,VACANT,NaN,NaN,FY12
1,16,520,"HCS Gulf Coast Veterans (Biloxi, MS)",Chief of Staff,"ROBERTS, SUSAN",5/1/16,NaN,FY11
2,10,539,"VAMC Cincinnati, OH",Chief of Staff,"PANOS, RALPH",2/25/16,NaN,FY13
3,11,550,"VAMC Danville, IL",Chief of Staff,VACANT,NaN,NaN,None
4,19,575,"VAMC Grand Junction, CO",Chief of Staff,"GINJUPALLI, SRINIVAS",3/7/16,NaN,FY12
5,11,610,HCS Northern Indiana (Ft. Wayne),Chief of Staff,VACANT,NaN,NaN,FY11
6,23,636,"HCS Nebraska-Western Iowa (Omaha, NE)",Chief of Staff,"BITTNER, MARVIN",4/3/16,636io,None
7,15,657,"HCS St. Louis, MO",Chief of Staff,"ADEGBOYEGA, PATRICK",12/4/15,657st,None
8,20,668,"VAMC Spokane, WA",Chief of Staff,"NYE, SCOTT",11/26/14,NaN,FY14
9,18,678,HCS Southern AZ (Tucson),Chief of Staff,VACANT,NaN,NaN,FY13
